# XGB FE & Opt/Tuning

This notebook offers:

- new iterations of feature engineering
- Param opt
- CV scoring

Credits to:
- https://www.kaggle.com/manels/lgb-starter (initial feature eng and starting notebook for dataloading)
- https://www.kaggle.com/ragnar123/optiver-realized-volatility-lgbm-baseline 

In [2]:
import os, gc
import glob

from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

from joblib import Parallel, delayed
from sklearn import preprocessing, model_selection

import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

%matplotlib inline

In [3]:
path_root = "./"
path_data = "./"

path_submissions = "./"

target_name = 'target'

In [4]:
train = pd.read_csv(os.path.join(path_data, 'train.csv'))

In [5]:
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [6]:
test = pd.read_csv(os.path.join(path_data,'test.csv'))

In [7]:
test.head()

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [8]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def count_unique(series):
    return len(np.unique(series))

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def rmspe(y_true, y_pred):
    return (np.sqrt(np.mean(np.square((y_true - y_pred)/y_true))))

def aggressor_side(series):
    return np.sign(series.diff())

def dollar_volume(df):
    return np.array(df['price'])*np.array(df['size'])

def signed_dollar_volume(df):
    side = aggressor_side(df)
    return np.array(df['dollar_volume'])*side

def kyle_lambda(df):
    diff = df['price'].diff().fillna(0)
    diff_sign = diff.apply(np.sign)
    diff_sign.replace(0,method='pad',inplace=True)
    signed_volume = df['size'] * diff_sign
#     print((diff / signed_volume).rolling(window=1).mean())
    return np.array((diff / signed_volume).rolling(window=1).mean())

def hasbrouck_lambda(log_ret, dollar_volume):
    log_ret_sign = log_ret.apply(np.sign).replace(0,method='pad')
    signed_dollar_volume_sqrt = np.sqrt(dollar_volume) * log_ret_sign
    
    return np.array((log_ret/signed_dollar_volume_sqrt).rolling(window=1).mean())

def vpin(volume, signed_buy_volume, window = 1):
    signed_sell_volume = volume - signed_buy_volume
    volume_imb = abs(signed_buy_volume - signed_sell_volume)
    return volume_imb.rolling(window=window).mean()/volume
   



In [9]:
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    df['side'] = np.sign(df['price'].diff())
    df['dollar_volume'] = df['price'] * df['size']
    df['signed_dollar_volume'] = df['dollar_volume'] * df['side']
    df['kyles'] = kyle_lambda(df)
    df['hasb'] = hasbrouck_lambda(df['log_return'],df['dollar_volume'])
    
    agg_dict = {
        'log_return':[realized_volatility],
        'size':[np.sum],
        'order_count':[np.mean],
        'dollar_volume':[np.sum],
        'signed_dollar_volume':[np.sum],
        'kyles':[np.mean],
        'hasb':[np.mean],
    }
    
    df_feature = df.groupby('time_id').agg(agg_dict)
    
    df_feature = df_feature.reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['trade_time_id_'],axis=1)
    
    return df_feature

In [50]:
file_path = os.path.join(path_data, 'trade_train.parquet/stock_id=0')
trade_preprocessor(file_path)

0


,trade_log_return_realized_volatility,trade_size_sum,trade_order_count_mean,trade_dollar_volume_sum,trade_signed_dollar_volume_sum,trade_kyles_mean,trade_hasb_mean,row_id
0,0.002006,3179,2.75,3190.139181,196.410613,0.000060,0.000096,0-5
1,0.000901,1289,1.9,1289.353432,197.051910,0.000098,0.000051,0-11
2,0.001961,2161,2.72,2158.608928,-1179.775465,0.000050,0.000090,0-16
3,0.001561,1962,3.933333,1959.605547,-307.434427,0.000026,0.000057,0-31
4,0.000871,1791,4.045455,1790.254496,-395.792862,0.000133,0.000056,0-62
...,...,...,...,...,...,...,...,...
3825,0.001519,3450,3.057692,3441.815546,120.949563,0.000040,0.000059,0-32751
3826,0.001411,4547,3.892857,4548.671493,643.305188,0.000063,0.000087,0-32753
3827,0.001521,4250,3.5,4247.563002,1651.284059,0.000065,0.000035,0-32758
3828,0.001794,3217,2.150943,3224.421796,-442.788901,0.000035,0.000053,0-32763


In [10]:
def book_preprocessor(file_path):
    df_book = pd.read_parquet(file_path)
    df_book['wap1'] = (df_book['bid_price1'] * df_book['ask_size1'] +
                                    df_book['ask_price1'] * df_book['bid_size1']) / (df_book['bid_size1'] + df_book['ask_size1'])
    df_book['wap2'] = (df_book['bid_price2'] * df_book['ask_size2'] +
                                    df_book['ask_price2'] * df_book['bid_size2']) / (df_book['bid_size2'] + df_book['ask_size2'])
    
    df_book['log_return1'] = df_book.groupby(by = ['time_id'])['wap1'].apply(log_return).fillna(0)
    df_book['log_return2'] = df_book.groupby(by = ['time_id'])['wap2'].apply(log_return).fillna(0)
    df_book['volume'] = (df_book['ask_size1'] + df_book['ask_size2']) + (df_book['bid_size1'] + df_book['bid_size2'])
    df_book['volume_imbalance'] = (df_book['ask_size1'] + df_book['ask_size2']) - (df_book['bid_size1'] + df_book['bid_size1'])
    
    agg_dict = {
        'log_return1':[realized_volatility],
        'log_return2':[realized_volatility],
        'volume':[np.sum],
        'volume_imbalance':[np.sum],
    }
    
    df_book = df_book.groupby(['time_id']).agg(agg_dict).reset_index()
    df_book.columns = ['_'.join(col) for col in df_book.columns]
    stock_id = file_path.split('=')[1]
    df_book['row_id'] = df_book['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_book = df_book.drop(['time_id_'],axis=1)
    
    return df_book

In [11]:
def get_stock_data(stock_ids,training=True):
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        if training:
            dataType = 'train'
            file_path_book = os.path.join(path_data, 'book_{}.parquet/stock_id={}'.format(dataType, stock_id))
            file_path_trade = os.path.join(path_data,'trade_{}.parquet/stock_id={}'.format(dataType, stock_id))
        else:
            dataType = 'test'
            file_path_book = os.path.join(path_data, 'book_{}.parquet/stock_id={}'.format(dataType, stock_id))
            file_path_trade = os.path.join(path_data,'trade_{}.parquet/stock_id={}'.format(dataType, stock_id))
            
        df_tmp = pd.merge(book_preprocessor(file_path_book),trade_preprocessor(file_path_trade),on='row_id',how='left')
        
        
        return pd.concat([df,df_tmp])
    
    
    df = Parallel(n_jobs=-1,verbose=1)(
            delayed(for_joblib)(stock_id) for stock_id in stock_ids
        )
            
        
    df = pd.concat(df,ignore_index=True)
    
    return df

In [53]:
stock_ids = [0,1]
get_stock_data(stock_ids)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    4.7s finished


,log_return1_realized_volatility,log_return2_realized_volatility,volume_sum,volume_imbalance_sum,row_id,trade_log_return_realized_volatility,trade_size_sum,trade_order_count_mean,trade_dollar_volume_sum,trade_signed_dollar_volume_sum,trade_kyles_mean,trade_hasb_mean
0,0.004499,0.006999,97696,2362,0-5,0.002006,3179,2.75,3190.139181,196.410613,0.000060,0.000096
1,0.001204,0.002476,82290,-26778,0-11,0.000901,1289,1.9,1289.353432,197.051910,0.000098,0.000051
2,0.002369,0.004801,78274,2524,0-16,0.001961,2161,2.72,2158.608928,-1179.775465,0.000050,0.000090
3,0.002574,0.003637,52232,2773,0-31,0.001561,1962,3.933333,1959.605547,-307.434427,0.000026,0.000057
4,0.001894,0.003257,60407,-18320,0-62,0.000871,1791,4.045455,1790.254496,-395.792862,0.000133,0.000056
...,...,...,...,...,...,...,...,...,...,...,...,...
7655,0.003723,0.004996,90929,1075,1-32751,0.001776,3249,2.77551,3248.982168,-432.521806,0.000053,0.000076
7656,0.010829,0.012168,280513,47142,1-32753,0.008492,75903,7.874317,76486.652579,18669.938909,0.000018,0.000049
7657,0.003135,0.004268,133527,-27629,1-32758,0.001927,2239,2.615385,2240.755934,-206.603570,0.000023,0.000056
7658,0.003750,0.005773,228422,10994,1-32763,0.002856,16648,2.93578,16696.638857,-1089.288566,0.000017,0.000039


In [12]:
def feval_RMSPE(preds, train_data):
    labels = train_data.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False


In [13]:
train_ids = train.stock_id.unique()

In [14]:
%%time
df_train=get_stock_data(train_ids)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   40.2s


Wall time: 2min 2s


[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:  2.0min finished


In [15]:
train['row_id'] = train['stock_id'].astype(str) +'-' +train['time_id'].astype(str)

train = train[['row_id','target']]

df_train = train.merge(df_train,on=['row_id'],how='left')

In [16]:
df_train.head()

,row_id,target,log_return1_realized_volatility,log_return2_realized_volatility,volume_sum,volume_imbalance_sum,trade_log_return_realized_volatility,trade_size_sum,trade_order_count_mean,trade_dollar_volume_sum,trade_signed_dollar_volume_sum,trade_kyles_mean,trade_hasb_mean
0,0-5,0.004136,0.004499,0.006999,97696,2362,0.002006,3179.0,2.75,3190.139181,196.410613,0.000060,0.000096
1,0-11,0.001445,0.001204,0.002476,82290,-26778,0.000901,1289.0,1.9,1289.353432,197.051910,0.000098,0.000051
2,0-16,0.002168,0.002369,0.004801,78274,2524,0.001961,2161.0,2.72,2158.608928,-1179.775465,0.000050,0.000090
3,0-31,0.002195,0.002574,0.003637,52232,2773,0.001561,1962.0,3.933333,1959.605547,-307.434427,0.000026,0.000057
4,0-62,0.001747,0.001894,0.003257,60407,-18320,0.000871,1791.0,4.045455,1790.254496,-395.792862,0.000133,0.000056


In [59]:
df_train.to_csv('./processed_train.csv')

In [17]:
test_ids = test.stock_id.unique()

In [18]:
%%time
df_test = get_stock_data(test_ids, training=False)

Wall time: 48.9 ms


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [19]:
df_test = test.merge(df_test,on=['row_id'],how='left')

In [20]:
df_train['stock_id'] = df_train['row_id'].apply(lambda x:x.split('-')[0])
df_test['stock_id'] = df_test['row_id'].apply(lambda x:x.split('-')[0])


stock_id_target_mean = df_train.groupby('stock_id')['target'].mean()
df_test['stock_id_target_enc'] = df_test['stock_id'].map(stock_id_target_mean)



In [21]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10,shuffle=True,random_state=729201)

tmp = np.repeat(np.nan,df_train.shape[0])

for idx1, idx2 in kf.split(df_train):
    target_mean = df_train.iloc[idx1].groupby('stock_id')['target'].mean()
    tmp[idx2] = df_train['stock_id'].iloc[idx2].map(target_mean)
    
df_train['stock_id_target_enc'] = tmp

In [73]:
df_train.head()

,row_id,target,log_return1_realized_volatility,log_return2_realized_volatility,volume_sum,volume_imbalance_sum,trade_log_return_realized_volatility,trade_size_sum,trade_order_count_mean,trade_dollar_volume_sum,trade_signed_dollar_volume_sum,trade_kyles_mean,trade_hasb_mean,stock_id,stock_id_target_enc
0,0-5,0.004136,0.004499,0.006999,97696,2362,0.002006,3179.0,2.75,3190.139181,196.410613,0.000060,0.000096,0,0.004026
1,0-11,0.001445,0.001204,0.002476,82290,-26778,0.000901,1289.0,1.9,1289.353432,197.051910,0.000098,0.000051,0,0.004022
2,0-16,0.002168,0.002369,0.004801,78274,2524,0.001961,2161.0,2.72,2158.608928,-1179.775465,0.000050,0.000090,0,0.003997
3,0-31,0.002195,0.002574,0.003637,52232,2773,0.001561,1962.0,3.933333,1959.605547,-307.434427,0.000026,0.000057,0,0.004034
4,0-62,0.001747,0.001894,0.003257,60407,-18320,0.000871,1791.0,4.045455,1790.254496,-395.792862,0.000133,0.000056,0,0.004026


In [22]:
df_train.tail()

,row_id,target,log_return1_realized_volatility,log_return2_realized_volatility,volume_sum,volume_imbalance_sum,trade_log_return_realized_volatility,trade_size_sum,trade_order_count_mean,trade_dollar_volume_sum,trade_signed_dollar_volume_sum,trade_kyles_mean,trade_hasb_mean,stock_id,stock_id_target_enc
428927,126-32751,0.003461,0.003691,0.005876,125874,-41748,0.002171,2570.0,2.783784,2568.838117,-14.933751,0.000085,0.000091,126,0.005317
428928,126-32753,0.003113,0.004104,0.004991,54261,-28823,0.002180,2323.0,3.418605,2327.828627,141.639961,0.000057,0.000088,126,0.005329
428929,126-32758,0.004070,0.003118,0.006019,89112,-79205,0.001921,3740.0,2.8,3742.254714,-2102.770354,0.000052,0.000084,126,0.005331
428930,126-32763,0.003357,0.003661,0.005362,170140,32288,0.002051,9389.0,2.925,9406.795437,2209.567113,0.000029,0.000048,126,0.005320
428931,126-32767,0.002090,0.002091,0.003037,115295,-55,0.001041,5325.0,3.0,5326.415054,-2655.876849,0.000006,0.000017,126,0.005320


In [23]:
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [24]:
df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)

In [25]:
df_train['trade_order_count_mean'] = df_train['trade_order_count_mean'].astype('float')

In [26]:
X = df_train.drop(['row_id','target'],axis=1)
y = df_train['target']

In [27]:
groupedKFold = PurgedGroupTimeSeriesSplit(n_splits=4, group_gap=12)
split = list(groupedKFold.split(df_train['target'],groups=df_train['stock_id'].values))

In [113]:
out_of_fold = pd.DataFrame()
models = []
scores = 0.0

In [35]:
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

class HOPT(object):
    
    def __init__(self,x_train,x_test,y_train,y_test):
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test 
        
    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status':STATUS_FAIL,
                   'exception':str(e)}
        
        return result, trials
    
    
    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg,para)
    
    def train_reg(self, reg, para):
        reg.fit(self.x_train,self.y_train,
               eval_set=[(self.x_train, self.y_train),(self.x_test,self.y_test)],
               **para['fit_params'])
        
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss':loss, 'status':STATUS_OK}
    
    

TODO: CV should occur w/n opt trials - move below cell into train func

In [38]:
from hyperopt import hp

model_name = 'xgb1'
pred_name = f'pred_{model_name}'


for _fold, (tr,te) in enumerate(split):
    print(f'Fold: {_fold}')
    
    trainSet_X = X.loc[tr]
    trainSet_Y = y.loc[tr]
    
    valSet_X = X.loc[te]
    valSet_Y = y.loc[te]
    
    weights = (1/np.square(trainSet_Y))
    valWeights = (1/np.square(valSet_Y))
    


    xgb_reg_params = {
        'tree_method':'gpu_hist',
        'learning_rate': hp.choice('learning_rate',np.arange(0.05,0.31,0.05)),
        'max_depth': hp.choice('max_depth',np.arange(5,16,1,dtype=int)),
        'min_child_weight': hp.choice('min_child_weight',np.arange(1,8,1,dtype=int)),
        'colsample_bytree': hp.choice('colsample_bytree',np.arange(0.3,0.8,0.1)),
        'subsample': hp.choice('subsample',np.arange(0.8,1)),
        'n_estimators': hp.choice('n_esitmators',np.arange(100,500,50))
    }

    xgb_fit_params = {
        'eval_metric':'rmse',
        'early_stopping_rounds': 10,
        'sample_weight_eval_set':[weights,valWeights],
        'sample_weight': weights,
        'verbose':False
    }

    xgb_params = {}
    xgb_params['reg_params'] = xgb_reg_params
    xgb_params['fit_params'] = xgb_fit_params
    xgb_params['loss_func'] = lambda y,pred: rmspe(y,pred)
    
    obj = HOPT(trainSet_X,valSet_X,trainSet_Y,valSet_Y)
    xgb_opt = obj.process(fn_name='xgb_reg',space=xgb_params,trials=Trials(),algo=tpe.suggest,max_evals=100)
    
    print('Model score: ', xgb_opt['loss'])
    

Fold: 0
 86%|████████████████████████████████████████▍      | 86/100 [10:53<01:46,  7.60s/trial, best loss: 0.4424418268176189]


KeyboardInterrupt: 

In [115]:
scores

0.29625